In [ ]:
#!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time

In [ ]:
def load_driver():
    url = #enter the url for the website
    driver = webdriver.Chrome(executable_path='path to chromedriver executable')
    driver.get(url)
    return (driver)

In [ ]:
#import the required packages 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common import exceptions
from selenium.webdriver.support.ui import Select


In [ ]:
#introduce wait for given time, check every 2 sec that given element has loaded in the DOM tree 
def waitForElementToLoadByType(driver,delay,by_type, elem_id):
    wait = WebDriverWait(driver,delay,poll_frequency=2)
    element = wait.until(EC.presence_of_element_located((by_type,elem_id)))
    return element

In [ ]:
def login_function():

    driver = load_driver()
    delay = 20
    driver.implicitly_wait(delay)
    
    email_elem = "userInput"
    next_click = "login-button"
    pass_elem = "okta-signin-password"
    login_elem ="okta-signin-submit"
    
    
    #send userid
    waitForElementToLoadByType(driver,delay,By.ID,email_elem).send_keys("enter user name")

    #click next
    waitForElementToLoadByType(driver,delay,By.ID,next_click).click()
    
    #wait
    driver.implicitly_wait(delay)

    # user name is already entered, provide password
    waitForElementToLoadByType(driver,delay,By.ID,pass_elem).send_keys("enter password")
    
    #wait
    driver.implicitly_wait(delay)
    
    # hit login button, login button may take time to load so wait - trying 10 times
    for i in range(0, 10):  
        try:  
            driver.execute_script('arguments[0].click()', waitForElementToLoadByType(driver,delay,By.ID,login_elem))
        except exceptions.StaleElementReferenceException as e:
            print(e)
            pass 
    
    return driver

In [ ]:
def pass_paramters_to_website(driver, theatre_val,country_val,city_val,zip_val,product_family_val,product_val,index):
    
    delay = 20
    try:
        # select ID1 from dropdown 
        select = Select(waitForElementToLoadByType(driver,delay,By.NAME,"ID1"))
        select.select_by_value(theatre_val)
        
        #implicit wait to allow element to load
        driver.implicitly_wait(delay)
        
        #select ID2 from dropdown
        select = Select(waitForElementToLoadByType(driver,delay,By.NAME,"ID2"))
        select.select_by_value(country_val)
        
        #implicit wait to allow element to load
        driver.implicitly_wait(delay)

        #enter ID3 in the textbox
        waitForElementToLoadByType(driver,delay,By.NAME,"ID3").send_keys(city_val)
        
        #implicit wait to allow element to load
        driver.implicitly_wait(delay)

        #enter ID4 in the textbox
        waitForElementToLoadByType(driver,delay,By.NAME,"ID4").send_keys(zip_val)
        
        #implicit wait to allow element to load
        driver.implicitly_wait(delay)

        #select ID5 from dropdown
        select = Select(waitForElementToLoadByType(driver,delay,By.NAME,"ID5"))
        select.select_by_value(product_family_val)
        
        time.sleep(5)

        #select ID6 from dropdown
        select = Select(waitForElementToLoadByType(driver,delay,By.NAME,"ID6"))
        select.select_by_value(product_val)
        
        #click submit button - 2 variations found in relative position, try both
        if(i==0):
            submit_elem = "enter XPATH name for submit element"
            element = waitForElementToLoadByType(driver,10,By.XPATH,submit_elem)
        else:
            submit_elem = "enter full XPATH for submit element"
            element = waitForElementToLoadByType(driver,20,By.XPATH,submit_elem)
        driver.execute_script('arguments[0].click()',element)
        time.sleep(5)

        #find the availability info Y/N in the table
        table_elem = "enter full XPATH name for table element here"
        availability = waitForElementToLoadByType(driver,delay,By.XPATH,table_elem).text
        print (index," ",city_val," ",zip_val," ",availability)
      
    except :
        availability = ' '
        print (index," ",city_val," ",zip_val," ",availability)
        
    return availability

In [ ]:
def clear_paramters_to_website(driver):
    
    delay = 20

    # wait for element to load, clear ID3
    waitForElementToLoadByType(driver,delay,By.NAME,"ID3").clear()
    
    #implicit wait
    driver.implicitly_wait(delay)

    # wait for element to load, clear ID4
    waitForElementToLoadByType(driver,delay,By.NAME,"ID4").clear()

    #implicit wait
    driver.implicitly_wait(delay)

In [ ]:
## write code to read csv file in a pandas dataframe, pass the params to function in a loop
import pandas as pd
import numpy as np
input_df = pd.read_excel('enter excel file location here', 
                         sheet_name= "Locations_BKP", 
                         dtype={'S NO':np.int32,'Address':"string",'City':"string",'State':"string",'Country':"string",'Zip':"string",'Type':"string","ProductName1":"string","ProductName2":"string"})
#make a copy of input dataframe
input1_df = input_df

#only US locations
input_df= input_df[input_df['Country'] == "USA"]

#row index in selected rows are not sequential, they do not start with zero so use reset index
input_df= input_df.reset_index(drop=True)
print(input_df.head())

In [ ]:
theatre_val = 'enter geogarphical theatre'
country_val = 'enter country ID'
product_family_val = 'enter product family ID'
product_val = 'enter product ID'

driver = login_function()

In [ ]:
#solve for i=0 case separately
i=0
city_val = input_df['City'][i]
zip_val = input_df['Zip'][i]
ProductName1_service_avail = ' '
product_family_val = 'enter product family ID'
product_val = 'enter product ID'
ProductName1_service_avail = pass_paramters_to_website(driver, theatre_val,country_val,city_val,zip_val,product_family_val,product_val,i)
input_df.loc[i,'ProductName1'] = ProductName1_service_avail

In [ ]:
size = len(input_df)
#solve for remaining items using a loop for ProductName1
for i in range (1,size):
    
    clear_paramters_to_website(driver)
    
    time.sleep(5)
    
    city_val = input_df['City'][i]
    zip_val = input_df['Zip'][i]
    ProductName1_service_avail = ' '
    product_family_val = 'enter product family ID'
    product_val = 'enter product ID'
    ProductName1_service_avail = pass_paramters_to_website(driver, theatre_val,country_val,city_val,zip_val,product_family_val,product_val,i)
    #write availability status to dataframe
    input_df.loc[i,'ProductName1'] = ProductName1_service_avail

In [ ]:
input2_df = input_df

#solve for ProductName2
for i in range (0,size):
    
        clear_paramters_to_website(driver)

        time.sleep(5)

        city_val = input_df['City'][i]
        zip_val = input_df['Zip'][i]
        
        ProductName2_service_avail = ' '
        product_family_val = 'enter product family ID'
        product_val = 'enter product ID'
        ProductName2_service_avail = pass_paramters_to_website(driver, theatre_val,country_val,city_val,zip_val,product_family_val,product_val,i)
        #write availability status to dataframe
        input_df.loc[i,'ProductName2'] = ProductName2_service_avail

In [ ]:
#write the dataframe to a csv file - drive:\\directory\file-name
input2_df.to_excel(r'enter path to the file', sheet_name='ScrapedData', index = False)